### Read file content and split into words
**Try reading file content:**

In [1]:
with open("sample_text.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of characters:", len(raw_text))
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


**Split text on whitespace, comman and period characters:**

In [2]:
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
# remove redundant whitespaces
preprocessed = [item for item in preprocessed if item.strip()]

print (preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### Convert tokens into token IDs
**Create a vocabulary**

In [3]:
all_tokens = sorted(list(set(preprocessed)))

all_tokens.extend([ "<|endoftext|>", "<|unk|>" ])

vacab_size = len(all_tokens)
print (vacab_size)

1132


In [4]:
vocab = { token:integer for integer, token in enumerate(all_tokens) }
for i, item in enumerate(vocab.items()):
    print (item)
    if i >= 20:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [5]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print (item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


**Implement a simple text tokenizer**

In [6]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        # create an inverse vocabulry that maps token ids back to the original text tokens
        self.int_to_str = { i:s for s, i in vocab.items() }

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        # process input text into token ids
        preprocessed = [ item.strip() for item in preprocessed if item.strip() ]
        # replace unknown words by <|unk|> tokens
        preprocessed = [ item if item in self.str_to_int else "<|unk|>" for item in preprocessed ]
        ids = [ self.str_to_int[s] for s in preprocessed ]
        return ids

    def decode(self, ids):
        # convert token ids back into text
        text = " ".join([ self.int_to_str[i] for i in ids ])
        # remove spaces before the specified punctuation
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


**Test the tokenizer**

In [7]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
       Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print (ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


**Turn the ids back to text**

In [8]:
print (tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [9]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))

print (tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [10]:
print (tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


### Byte pair encoding


In [11]:
%pip install tiktoken


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
from importlib.metadata import version
import tiktoken

print ("tiktoken version:", version("tiktoken"))

tiktoken version: 0.12.0


In [13]:
tokenizer = tiktoken.get_encoding("gpt2")

text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces of the someunknownplace.")

integers = tokenizer.encode(text, allowed_special={ "<|endoftext|>" })

print (integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 262, 617, 34680, 5372, 13]


**Convert token IDs back to text**

In [14]:
strings = tokenizer.decode(integers)
print (strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the someunknownplace.


Try the BPE tokenizer from the tiktoken library on the unknown words “Akwirw ier” and
print the individual token IDs. Then, call the decode function on each of the resulting
integers in this list to reproduce the mapping shown in figure 2.11. Lastly, call the
decode method on the token IDs to check whether it can reconstruct the original
input, “Akwirw ier.”

In [15]:
integers = tokenizer.encode("Akwirw ier")
print (integers)

[33901, 86, 343, 86, 220, 959]


In [16]:
strings = tokenizer.decode(integers)
print (strings)

Akwirw ier


In [17]:
for token in integers:
    print ("" + str(token) + " == \"" + tokenizer.decode([token]) + "\"")

33901 == "Ak"
86 == "w"
343 == "ir"
86 == "w"
220 == " "
959 == "ier"


Byte pair encoding builds its vocabulary by iteratively merging frequent characters into subwords and frequent subwords into words. For example, BPE starts with adding all individual single characters to its vocabulary (“a”,
 “b,” etc.). In the next stage, it merges character combinations that frequently occur together into subwords. For example, “d” and “e” may be merged into the subword “de,” which is common in many English words like “define”,  “depend,” “made,” and “hidden.” The merges are determined by a frequency cutoff.

### Data sampling with a sliding window

In [20]:
enc_text = tokenizer.encode(raw_text)
print (len(enc_text))

5145


In [22]:
# remove the first 50 tokens from the dataset for demonstration purposes,
# as it results in a slightly more interesting text passage in the next steps:
enc_sample = enc_text[50:]

One of the easiest and most intuitive ways to create the input–target pairs for the next-
word prediction task is to create two variables, x and y, where x contains the input
tokens and y contains the targets, which are the inputs shifted by 1:

In [23]:
# the context size determines how many tokens are included in the input
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


By processing the inputs along with the targets, which are the inputs shifted by one
position, we can create the next-word prediction tasks (see figure 2.12), as follows:

In [24]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


Everything left of the arrow (---->) refers to the input an LLM would receive, and
the token ID on the right side of the arrow represents the target token ID that the
LLM is supposed to predict. Let’s repeat the previous code but convert the token IDs
into text:

In [25]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


We’ve now created the input–target pairs that we can use for LLM training.

### Implement efficient data loader that iterates over the input dataset and returns the inpuit and targes as PyTorch tensors

In [26]:
%pip install torch numpy


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
import torch
